In [ ]:
## term のLDA モデルを使ったエンコード

import pprint as pp
## get_term_topics(..) では　minimu_probability = 0 としてもprobabaly = 0 の topic IDs が
## 得られないので，sparse encoding しか得られない
pair_term_sparse_enc = { term : { x[0] : x[1] for x
                                 in pair_lda.get_term_topics(tid, minimum_probability = 0) }
                       for tid, term in pair_diction.items() }
## check
pp.pprint(random.sample(sorted(pair_term_sparse_enc.items()), 3))
print(f"Number of terms: {len(pair_term_sparse_enc)}")

In [ ]:
## Pandas を使って sparse_enc を full enc に変換
pair_term_enc_df = pd.DataFrame.from_dict({ k : pd.Series(v) for k, v in pair_term_sparse_enc.items()})
## 上で生じたNaN を0に変換
pair_term_enc_df = pair_term_enc_df.fillna(0)    
## データを転地
pair_term_enc_df = pair_term_enc_df.transpose()
pair_term_enc_df

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.hist(pair_term_enc_df.sum(axis = 1))
plt.show()

In [ ]:
## density で filtering
import heapq
def nth_largest(n: int, numbers: list):
    return heapq.nlargest(n, numbers)[-1]
#
threshold = 100
lowest_density = nth_largest(threshold, pair_term_enc_df.sum(axis = 1))
size0 = len(pair_term_enc_df)
pair_term_enc_df_filtered = pair_term_enc_df[ pair_term_enc_df.sum(axis = 1) > lowest_density ]
size1 = len(pair_term_enc_df_filtered)
print(f"{size1} rows remain, discarding {size0 - size1} rows")

In [ ]:
## sampling term_enc_df for hc
term_hc_sampling_rate = 1.0
sampled_pair_term_enc_df = \
    pair_term_enc_df_filtered.sample(round(len(pair_term_enc_df_filtered) * term_hc_sampling_rate))
#
sampled_pair_term_enc_df.sample(10)

In [ ]:
## term = pair の階層クラスタリングの実行
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

## 日本語表示のための設定
#plt.rcParams["font.family"] = "Hiragino sans" # Windows/Linux では別のフォントを指定
## 変数名の変更
hc_df = sampled_pair_term_enc_df

## 描画サイズの指定
plt.figure(figsize = (6, round(10 * len(hc_df) * 0.017))) # This needs to be run here, before dendrogram construction.

## 距離行列の構築
linkage_methods = [ 'centroid', 'median', 'ward' ]
linkage_method = linkage_methods[-1]
#pair_linkage = linkage(hc_df, method = linkage_method, metric = 'euclidean')
pair_linkage = linkage(hc_df, metric = 'euclidean')

## 事例ラベルの生成
max_term_length = max([ len(x) for x in list(hc_df.index)])
label_vals = [ x[:max_term_length] for x in list(hc_df.index) ] # truncate doc keys
## 樹状分岐図の作成
dendrogram(pair_linkage, orientation = 'left', labels = label_vals, leaf_font_size = 8)

## 題の指定
term_df_size = len(hc_df)
hc_sampling_rate = f"{100 * term_hc_sampling_rate:.2f}"
#domain_names = ', '.join(selected_domains.keys())
title_header = f"Hierarchical clustering of {term_df_size} terms (= {hc_sampling_rate}% sample) via\n"
title_body = f"LDA ({n_topics} topics; term: {sp_term_type}-{sn_term_type} pair)"
title_val = title_header + title_body
plt.title(title_val)
plt.show()